In [1]:
def read_file(file):
    import csv
    with open(file, 'r') as f:
        reader = csv.reader(f, delimiter=',')
        header = next(reader)
        contacts = [[line[0], line[1]] for line in reader]
    return contacts

In [2]:
def read_template(message):
    from string import Template
    with open(message, 'r', encoding='utf-8') as f:
        content = f.read()
    return Template(content)

In [5]:
def send_message(contact_file, message_file):
    
    recipients = read_file(contact_file)
    message_template = read_template(message_file)
    
    import smtplib
    from mail_data import EMAIL, PASS
    
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(EMAIL, PASS)
    
    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText

    for contact in recipients:
        name = contact[0]
        email = contact[1]

        message = MIMEMultipart()

        named_template = message_template.substitute(RECIPIENT_NAME=name.title())

        message['From'] = 'somemail@mail.com'
        message['To'] = email
        message['Subject'] = "Subject"

        message.attach(MIMEText(named_template, 'plain'))
        server.send_message(message)

        del message
    
    server.quit()


In [4]:
send_message('contacts.csv', 'message.txt')